## Parse PCAP files and find out which IP address corresponds to which MAC address

When multiple IP addresses are assigned to the same MAC, it will report an warning.

In [ ]:
import dpkt
import time
import socket
import os
import pandas as pd
import numpy as np
import struct
from IPython.display import display

### constants
g_dataDir = '../iot_data/'
g_dataExtension = '.pcap'
g_testfile = '../iot_data/16-09-27.pcap'
g_lanIpStartsWith = '192.168.'
g_showProgressPacketInterval = 1000000
g_device_info = {
    'd0:52:a8:00:67:5e': ['Smart Things', 'Wired'],
    '44:65:0d:56:cc:d3': ['Amazon Echo', 'Wireless'],
    '70:ee:50:18:34:43': ['Netatmo Welcome', 'Wireless'],
    'f4:f2:6d:93:51:f1': ['TP-Link Day Night Cloud camera', 'Wireless'],
    '00:16:6c:ab:6b:88': ['Samsung SmartCam', 'Wireless'],
    '30:8c:fb:2f:e4:b2': ['Dropcam', 'Wireless'],
    '00:62:6e:51:27:2e': ['Insteon Camera', 'Wired '],
    'e8:ab:fa:19:de:4f': [ 'Wireless'],
    '00:24:e4:11:18:a8': ['Withings Smart Baby Monitor', 'Wired'],
    'ec:1a:59:79:f4:89': ['Belkin Wemo switch', 'Wireless'],
    '50:c7:bf:00:56:39': ['TP-Link Smart plug', 'Wireless'],
    '74:c6:3b:29:d7:1d': ['iHome', 'Wireless'],
    'ec:1a:59:83:28:11': ['Belkin wemo motion sensor', 'Wireless'],
    '18:b4:30:25:be:e4': ['NEST Protect smoke alarm', 'Wireless'],
    '70:ee:50:03:b8:ac': ['Netatmo weather station', 'Wireless'],
    '00:24:e4:1b:6f:96': ['Withings Smart scale', 'Wireless'],
    '74:6a:89:00:2e:25': ['Blipcare Blood Pressure meter', 'Wireless'],
    '00:24:e4:20:28:c6': ['Withings Aura smart sleep sensor', 'Wireless'],
    'd0:73:d5:01:83:08': ['Light Bulbs LiFX Smart Bulb', 'Wireless'],
    '18:b7:9e:02:20:44': ['Triby Speaker', 'Wireless'],
    'e0:76:d0:33:bb:85': ['PIX-STAR Photo-frame', 'Wireless'],
    '70:5a:0f:e4:9b:c0': ['HP Printer', 'Wireless'],
    '08:21:ef:3b:fc:e3': ['Samsung Galaxy Tab', 'Wireless'],
    '30:8c:fb:b6:ea:45': ['Nest Dropcam', 'Wireless'],
    '40:f3:08:ff:1e:da': ['Android Phone', 'Wireless'],
    '74:2f:68:81:69:42': ['Laptop', 'Wireless'],
    'ac:bc:32:d4:6f:2f': ['MacBook', 'Wireless'],
    'b4:ce:f6:a7:a3:c2': ['Android Phone', 'Wireless'],
    'd0:a6:37:df:a1:e1': ['IPhone', 'Wireless'],
    'f4:5c:89:93:cc:85': ['MacBook/Iphone', 'Wireless'],
    '14:cc:20:51:33:ea': ['TPLink Router Bridge LAN (Gateway)', 'Wired']
}

### initialization
devices = {}   # key: MAC, value: IP
start = time.time()
dataFiles = []
for fileName in os.listdir(g_dataDir):
    if fileName.endswith(g_dataExtension):
        dataFiles.append(os.path.join(g_dataDir, fileName))

### parse pcap files
for fileName in dataFiles:
    file = open(fileName,'rb')
    counter = 0
    for ts, pkt in dpkt.pcapng.Reader(file):
        counter += 1
        if counter % g_showProgressPacketInterval == 0:
            print(counter, 'packets processed')
            
        eth = dpkt.ethernet.Ethernet(pkt)
        if eth.type != dpkt.ethernet.ETH_TYPE_ARP:
            continue
            
        arp = eth.data
        ip = socket.inet_ntoa(arp.spa)
        if not ip.startswith(g_lanIpStartsWith):
            continue
            
        mac = "%02x:%02x:%02x:%02x:%02x:%02x" % struct.unpack("BBBBBB",arp.sha)
        if mac not in devices:
            devices[mac] = ip
            print(mac, ip)
        else:
            if devices[mac] != ip:
                print('Warning: mac=%s, ip1=%s, ip2=%s' % (mac, devices[mac], ip) )
    
    print('Finished parsing: ' + fileName + '\n\n')

print ("Time elapsed: " + str(time.time() - start))
for key, value in devices.items():
    if key in g_device_info:
        print(key, value, g_device_info[key][0], g_device_info[key][1])
    else:
        print(key, value)
        

14:cc:20:51:33:ea 192.168.1.1
44:65:0d:56:cc:d3 192.168.1.240
70:ee:50:18:34:43 192.168.1.241
70:5a:0f:e4:9b:c0 192.168.1.236
70:ee:50:03:b8:ac 192.168.1.112
d0:52:a8:00:67:5e 192.168.1.196
00:24:e4:11:18:a8 192.168.1.166
30:8c:fb:2f:e4:b2 192.168.1.106
ec:1a:59:83:28:11 192.168.1.193
f4:f2:6d:93:51:f1 192.168.1.143
ec:1a:59:79:f4:89 192.168.1.223
00:16:6c:ab:6b:88 192.168.1.249
18:b7:9e:02:20:44 192.168.1.120
50:c7:bf:00:56:39 192.168.1.227
e0:76:d0:33:bb:85 192.168.1.177
08:21:ef:3b:fc:e3 192.168.1.239
18:b4:30:25:be:e4 192.168.1.168
00:24:e4:1b:6f:96 192.168.1.238
Finished parsing: ../iot_data/16-09-24.pcap


00:62:6e:51:27:2e 192.168.1.192
d0:73:d5:01:83:08 192.168.1.118
00:24:e4:20:28:c6 192.168.1.156
74:2f:68:81:69:42 192.168.1.208
40:f3:08:ff:1e:da 192.168.1.180
ac:bc:32:d4:6f:2f 192.168.1.243
74:c6:3b:29:d7:1d 192.168.1.163
00:24:e4:10:ee:4c 192.168.1.220
b4:ce:f6:a7:a3:c2 192.168.1.248
30:8c:fb:b6:ea:45 192.168.1.237
1000000 packets processed
Finished parsing: ../iot_data/16-1